In [43]:
import pandas as pd

In [44]:
df_15 = pd.read_csv('no2_2015.csv')
df_16 = pd.read_csv('no2_2016.csv')
df_17 = pd.read_csv('no2_2017.csv')
df_18 = pd.read_csv('no2_2018.csv')
df_19 = pd.read_csv('no2_2019.csv')
df_20 = pd.read_csv('no2_2020.csv')
df_21 = pd.read_csv('no2_2021.csv')
df_22 = pd.read_csv('no2_2022.csv')
pdList = [df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22 ]
df = pd.concat(pdList)
df = df.reset_index().drop('index', axis = 1)
df['date'] =  df['MTU (UTC)'].str[:16]
df = df.rename(columns={'Day-ahead Price [EUR/MWh]':'price_no2'})[['date','price_no2']]
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')

In [45]:
## Make year as a column
df['year'] = df['date'].dt.year
# Find mowing average with window size = 876
df['moving_average'] = df['price_no2'].rolling(window=876).mean()
#find mean of each year
df_mean = df.groupby('year').mean('price_no2')
df_mean = df_mean.reset_index()[['year','price_no2']]
df_moving = df.groupby('year').min('moving_average').reset_index()[['year','moving_average']]
#merge two dataframes
df2 = pd.merge(df_mean, df_moving, on='year')
# Make a new column "capture_nuclear" with formula (price_no2 - moving_average*0.1)/0.9
df2['capture_nuclear'] = (df2['price_no2'] - df2['moving_average']*0.1)/0.9
# rename price_no2 to mean_price_no2
df2 = df2.rename(columns={'price_no2':'mean_price_no2'})
df2[['year','mean_price_no2','capture_nuclear','moving_average']]

,year,mean_price_no2,capture_nuclear,moving_average
0,2015,19.822256,21.062201,8.662751
1,2016,25.151584,25.991791,17.589715
2,2017,28.831518,29.405909,23.661998
3,2018,43.252344,44.660657,30.577523
4,2019,39.264231,40.323860,29.727568
5,2020,9.290370,10.172486,1.351324
6,2021,75.107840,81.389930,18.569030
7,2022,211.277426,225.030545,87.499349


In [46]:
## year to date 2023
df = pd.read_csv('no2_2023.csv')
df['date'] =  df['MTU (CET/CEST)'].str[:16]
df = df.rename(columns={'Day-ahead Price [EUR/MWh]':'price_no2'})[['date','price_no2']]
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
# Remove rows where "price_no2" equal -
df = df[df['price_no2'] != '-']
df['price_no2'] = df['price_no2'].astype(float)
# Find moving average with window size = 10% * hours to date
n = round(df.shape[0]*0.1,0)
df['moving_average'] = df['price_no2'].rolling(window=876).mean()
min_moving_average = df['moving_average'].min()
print(f'''Mean price NO2 2023 until 22th of may {round(df['price_no2'].mean(),2)}, capture rate of nuclear {round((df['price_no2'].mean() - min_moving_average*0.1)/0.9,2)}''')

Mean price NO2 2023 until 22th of may 100.45, capture rate of nuclear 102.32
